In [1]:
import os
import sys
from glob import glob
import torch
import cv2

from model.autoencoder import AutoEncoder
from utils.dataset import Screen
from utils.config import cm

In [2]:
cm.load('/Users/laiyuchun/Desktop/Autoencoder/setting.cfg')
cm['ROOT']['DATASET']
dataset_root = os.path.join(os.getcwd(), cm['ROOT']['DATASET'])
para_root = os.path.join(os.getcwd(), cm['ROOT']['PARA'])
test_case_root = os.path.join(os.getcwd(), cm['ROOT']['TEST_CASE'])
print(dataset_root, para_root, test_case_root)



/Users/laiyuchun/Desktop/Autoencoder/dataset /Users/laiyuchun/Desktop/Autoencoder/para/autoencoder_v2.pt /Users/laiyuchun/Desktop/Autoencoder/test_case


In [5]:
regexp = os.path.join(dataset_root, '*.png')

screen_data = Screen(regexp)
print(screen_data.__len__())
print(screen_data.__getitem__(5))

1350
(tensor([[[0.2863, 0.2902, 0.2510,  ..., 0.3357, 0.3024, 0.3150],
         [0.3442, 0.3005, 0.3078,  ..., 0.5095, 0.3502, 0.4688],
         [0.4649, 0.3318, 0.4504,  ..., 0.4897, 0.6152, 0.6583],
         ...,
         [0.4099, 0.4469, 0.4198,  ..., 0.0987, 0.4249, 0.0718],
         [0.1227, 0.3811, 0.0826,  ..., 0.6311, 0.5878, 0.6231],
         [0.1854, 0.1710, 0.1645,  ..., 0.4118, 0.4405, 0.4235]],

        [[0.4118, 0.4405, 0.4235,  ..., 0.0813, 0.4330, 0.0656],
         [0.1192, 0.3939, 0.0819,  ..., 0.6674, 0.6046, 0.6425],
         [0.1854, 0.1697, 0.1721,  ..., 0.4150, 0.4328, 0.4235],
         ...,
         [0.5686, 0.5804, 0.5647,  ..., 0.8745, 0.8667, 0.8706],
         [0.2667, 0.2275, 0.2275,  ..., 0.5529, 0.5647, 0.5647],
         [0.5529, 0.5647, 0.5620,  ..., 0.5686, 0.5804, 0.5647]],

        [[0.5686, 0.5804, 0.5647,  ..., 0.8745, 0.8667, 0.8706],
         [0.2667, 0.2275, 0.2275,  ..., 0.5529, 0.5647, 0.5647],
         [0.5529, 0.5647, 0.5620,  ..., 0.5686, 0.58

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoEncoder(3, 2, mode='linear').to(device)

In [7]:
with torch.no_grad():
    tmp = torch.randn(1, 3, 256, 256).to(device)
    test_pred = model(tmp)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (768x256 and 3x128)

In [32]:
#TODO: add inference
model = torch.load(para_root, map_location=device)
optimizer = torch.optim.Adam(
    model.parameters(), lr=1e-3, weight_decay=1e-5)
loss_F = torch.nn.MSELoss()

In [36]:
files = glob(os.path.join(test_case_root, '*'))
files

['/Users/laiyuchun/Desktop/Autoencoder/test_case/camera_shift.png',
 '/Users/laiyuchun/Desktop/Autoencoder/test_case/screen_change.png',
 '/Users/laiyuchun/Desktop/Autoencoder/test_case/normal.png']

In [39]:
tmp = {}
for file in files:
    img = cv2.imread(file)
    img = img[200:800, 250:600]
    img = img / 255
    img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)
    img = torch.Tensor(img).reshape(1, 3, 256, 256)
    pred = model(img)
    loss = loss_F(img, pred)
    tmp[file] = loss

tmp

{'/Users/laiyuchun/Desktop/Autoencoder/test_case/camera_shift.png': tensor(0.0044, grad_fn=<MseLossBackward>),
 '/Users/laiyuchun/Desktop/Autoencoder/test_case/screen_change.png': tensor(0.0050, grad_fn=<MseLossBackward>),
 '/Users/laiyuchun/Desktop/Autoencoder/test_case/normal.png': tensor(0.0026, grad_fn=<MseLossBackward>)}